**Data Manipulation Language (DML)**: SELECT, INSERT, UPDATE, DELETE, TRUNCATE, MERGE.
**Data Modification**: a subset of DML. INSERT, UPDATE, DELETE, TRUNCATE, MERGE. Everything except for SELECT.

Inserting Data: INSERT VALUES, INSERT SELECT, INSERT EXEC, SELECT INTO, BULK INSERT

**INSERT VALUES**

Inserts rows into a table based on specified values.

In [44]:
use Northwinds2022TSQLV7
ALTER TABLE dbo.OrderDetails
DROP CONSTRAINT FK_OrderDetails_Orders;

DROP TABLE IF EXISTS dbo.Orders;

CREATE TABLE dbo.Orders
(
	orderid INT NOT NULL
		CONSTRAINT PK_Orders PRIMARY KEY,
	orderdate DATE NOT NULL
		CONSTRAINT DFT_orderdate DEFAULT (SYSDATETIME()),
	empid INT NOT NULL,
	custid	VARCHAR (10) NOT NULL 

);

INSERT INTO dbo.Orders (orderid, orderdate, empid, custid)
	VALUES (10001,'20160212', 5, 'A');

SELECT * FROM dbo.orders
-- if column is not specified, default value will be used. If no default value, NULL.
INSERT INTO dbo.Orders (orderid, empid, custid)
	VALUES (10002, 5, 'B');

SELECT * FROM dbo.orders

-- enhanced insert for multiple inserts.
INSERT INTO dbo.Orders 
 (orderid, orderdate, empid, custid)
VALUES 
  (10003, '20160213', 4, 'B'),
  (10004, '20160214', 1, 'A'),
  (10005, '20160213', 1, 'C'),
  (10006, '20160215', 3, 'C');

SELECT * FROM dbo.orders

--You can make a derived table from enhanced insert. Following the values, you designate o for the table alias and column aliases.
SELECT *
	FROM (VALUES 
			(10003, '20160213', 4, 'B'),
			  (10004, '20160214', 1, 'A'),
			  (10005, '20160213', 1, 'C'),
			  (10006, '20160215', 3, 'C'))
	AS o (orderid, orderdate, empid, customerId);

(1 row affected)

(1 row affected)

(1 row affected)

(2 rows affected)

(4 rows affected)

...

Total execution time: 00:00:00.091

orderid,orderdate,empid,custid
10001,2016-02-12,5,A


orderid,orderdate,empid,custid
10001,2016-02-12,5,A
10002,2022-11-02,5,B


orderid,orderdate,empid,custid
10001,2016-02-12,5,A
10002,2022-11-02,5,B
10003,2016-02-13,4,B
10004,2016-02-14,1,A
10005,2016-02-13,1,C
10006,2016-02-15,3,C


orderid,orderdate,empid,customerId
10003,20160213,4,B
10004,20160214,1,A
10005,20160213,1,C
10006,20160215,3,C


**INSERT SELECT** 

Inserts a set of rows returned by a SELECT query into a target table.

In [45]:
use Northwinds2022TSQLV7
INSERT INTO dbo.Orders (orderid, orderdate, empid, custid)
	SELECT orderid, orderdate, EmployeeId, CustomerId
	FROM sales.[ORDER]
	WHERE ShipToCountry = N'UK';

--INSERT SELECT is performed as a transaction. If any row fails to entere the target table, none of the rows enteres the table.
SELECT * FROM dbo.orders

(56 rows affected)

(62 rows affected)

Total execution time: 00:00:00.020

orderid,orderdate,empid,custid
10001,2016-02-12,5,A
10002,2022-11-02,5,B
10003,2016-02-13,4,B
10004,2016-02-14,1,A
10005,2016-02-13,1,C
10006,2016-02-15,3,C
10289,2014-08-26,7,11
10315,2014-09-26,4,38
10318,2014-10-01,8,38
10321,2014-10-03,3,38


INSERT EXEC

You use this to insert a result set returned from a stored prcedure or a dynamic SQL batch into a target table. Stored procedure is a group of saved SQL statements. It is different from View in that View acts like a table while stored procedure can do multiple statements like insert, delete, etc.

In [46]:
use Northwinds2022TSQLV7
DROP PROC IF EXISTS SALES.GetOrders;
GO

CREATE PROC SALES.GetOrders
	@country AS NVARCHAR (40)

AS 

	SELECT OrderId, OrderDate, EmployeeId, CustomerId
	FROM sales.[Order]
	WHERE ShipToCountry = @country;

GO 

EXEC sales.GetOrders @country = N'France' -- nvarchar(40)

INSERT INTO dbo.Orders (orderId, orderdate, empid, custid)
	EXEC sales.GetOrders @country = N'France' -- nvarchar(40)

Commands completed successfully.

Commands completed successfully.

(77 rows affected)

(77 rows affected)

Total execution time: 00:00:00.076

OrderId,OrderDate,EmployeeId,CustomerId
10248,2014-07-04,5,85
10251,2014-07-08,3,84
10265,2014-07-25,2,7
10274,2014-08-06,6,85
10295,2014-09-02,2,85
10297,2014-09-04,5,7
10311,2014-09-20,1,18
10331,2014-10-16,9,9
10334,2014-10-21,8,84
10340,2014-10-29,1,9


**SELECT INTO**
Copies the entire table data into a target table. It is a nonstandard T-SQL statement. It copies from the source table column names, types, nullability, identity property and the data. It does not copy constraints, indexes, triggers, column properties such as SPARSE and FILESTREAM, and permissions. It is efficient because it uses minimal logging.

In [48]:
use Northwinds2022TSQLV7
DROP TABLE IF EXISTS dbo.Locations;

SELECT c.CustomerCountry, c.CustomerRegion, c.CustomerCity 
INTO dbo.Locations --it creates a brand new table without the CREATE clause
FROM sales.Customer AS c

select * from dbo.Locations;
--If you want to use SELECT INTO with SET OPERATORS, you specifiy the INTO clause right in front of the FROM caluse of the FIRST query.

DROP TABLE IF EXISTS dbo.locations;

SELECT Customercountry, Customerregion, CustomerCity
INTO dbo.Locations
FROM sales.Customer

EXCEPT

SELECT EmployeeCountry, EmployeeRegion, EmployeeCity
FROM HumanResources.Employee 

select * from dbo.Locations;

(91 rows affected)

(91 rows affected)

(66 rows affected)

(66 rows affected)

Total execution time: 00:00:00.078

CustomerCountry,CustomerRegion,CustomerCity
Germany,NULL,Berlin
Mexico,NULL,México D.F.
Mexico,NULL,México D.F.
UK,NULL,London
Sweden,NULL,Luleå
Germany,NULL,Mannheim
France,NULL,Strasbourg
Spain,NULL,Madrid
France,NULL,Marseille
Canada,BC,Tsawassen


Customercountry,Customerregion,CustomerCity
Argentina,NULL,Buenos Aires
Austria,NULL,Graz
Austria,NULL,Salzburg
Belgium,NULL,Bruxelles
Belgium,NULL,Charleroi
Brazil,RJ,Rio de Janeiro
Brazil,SP,Campinas
Brazil,SP,Resende
Brazil,SP,Sao Paulo
Canada,BC,Tsawassen


**BULK INSERT**

Inserts data from a file into an existing table.

In [11]:
use Northwinds2022TSQLV7
BULK INSERT dbo.Orders FROM 'D:\School\QC\Data\Database\Chapter 8 Data modification\orders.txt'
	WITH 
		(
			DATAFILETYPE = 'char',
			FIELDTERMINATOR = ',',
			ROWTERMINATOR= '\n'

		)

: Msg 4860, Level 16, State 1, Line 2
Cannot bulk load. The file "D:\School\QC\Data\Database\Chapter 8 Data modification\orders.txt" does not exist or you don't have file access rights.

Total execution time: 00:00:00.020

**The identity property and the sequence object**

  

IDENTITY COLUMN property and SEQUENCE OBJECT are two SQL Server's built-in solutions to automatically generate numeric keys.

**IDENTITY**

  

Identity is a standard column propety. You can define this property for a column with any numeric type with a scale of zero (no fraction/decimal). You use this propety to generate surrogate keys, which are keys produced by the system.  
Seed: first value  
Increment: step value

In [49]:
use Northwinds2022TSQLV7
DROP TABLE IF EXISTS dbo.T1;

CREATE TABLE dbo.T1
(
	keycol INT NOT NULL IDENTITY (1,1)
		CONSTRAINT PK_T1 PRIMARY KEY,
	datacol VARCHAR(10) NOT NULL 
		CONSTRAINT CHK_T1_datacol CHECK (datacol LIKE '[ABCDEFGHIJKLMNOPQRSTVQWZYZ]%')
)

--In INSERT statements, you must ignore the identity column. It gets generated automatically.

INSERT INTO dbo.T1 (datacol) VALUES ('AAAAA'), ('CCCCC'), ('BBBBB');
SELECT * FROM dbo.T1 AS t

--to refer to the identity column, you can specify the column name like this
SELECT t.keycol FROM dbo.T1 AS t
--or you can use this generic form
SELECT $identity FROM dbo.T1 AS t



(3 rows affected)

(3 rows affected)

(3 rows affected)

(3 rows affected)

Total execution time: 00:00:00.032

keycol,datacol
1,AAAAA
2,CCCCC
3,BBBBB


keycol
1
2
3


keycol
1
2
3


<span style="color: #608b4e;">--@@identity&nbsp;and&nbsp;SCOPE_IDENTITY&nbsp;returns&nbsp;last&nbsp;identity&nbsp;value&nbsp;produced&nbsp;by&nbsp;current&nbsp;session.&nbsp;</span> 

<span style="color: #dcdcaa;">scope_identity</span>  <span style="color: #dcdcaa;">@@identity</span>  IDENT\_CURRENT

<span style="color: #b5cea8;">4</span>                   <span style="color: #b5cea8;">4</span>           <span style="color: #b5cea8;">4</span>

<span style="color: #608b4e;">IDENT_CURRENT&nbsp;returns&nbsp;last&nbsp;identity&nbsp;value&nbsp;regardless&nbsp;of&nbsp;session (in a new query window)</span>

scope\_identity @@identity IDENT\_CURRENT

> NULL NULL 4

**Drawbacks of Identity Property**

<span style="color: #608b4e;">Use&nbsp;the&nbsp;identity&nbsp;property&nbsp;only&nbsp;if&nbsp;you&nbsp;can&nbsp;allow&nbsp;gaps&nbsp;between&nbsp;the&nbsp;keys.</span>

In [50]:
use Northwinds2022TSQLV7
--The following insert fails. The identity value changed to 5. Next successful run will begin with 6. 
INSERT INTO dbo.T1(datacol) VALUES ('12345')

INSERT INTO dbo.T1(datacol) VALUES ('EEEEE')

SELECT * FROM DBO.T1;

--Unclean terminations such as power failures can result in gaps between the keys. You cannot add identity property to an existing column or remove it from an existing column. 
--you can specify your own explicit values for the identity column when you insert rows. But you cannot update an identity column.

SET IDENTITY_INSERT dbo.T1 ON;
INSERT INTO dbo.T1(keycol, datacol) VALUES (4, 'FFFFF')
SET IDENTITY_INSERT dbo.T1 OFF;
SELECT * FROM dbo.T1

--If you run the IDENT_CURRENT function now, it will be 6 not 5. Even though you just produced 5. 
--The current idenity value changes only if the explicit value you provided is greater than the current identity value.

INSERT INTO dbo.T1 (datacol) VALUES ('GGGGG')

SELECT * FROM dbo.T1

--Identity property does not enforce uniqueness. When IDENTITY_INSERT is ON, you can insert values that already exist in the table. 
--In order to guarantee uniqueness, you must define primary key or unqiue constraint on that column.

The statement has been terminated.

(1 row affected)

(4 rows affected)

(1 row affected)

(1 row affected)

(6 rows affected)

Total execution time: 00:00:00.043

keycol,datacol
1,AAAAA
2,CCCCC
3,BBBBB
5,EEEEE


keycol,datacol
1,AAAAA
2,CCCCC
3,BBBBB
4,FFFFF
5,EEEEE
6,GGGGG


: Msg 547, Level 16, State 0, Line 3
The INSERT statement conflicted with the CHECK constraint "CHK_T1_datacol". The conflict occurred in database "Northwinds2022TSQLV7", table "dbo.T1", column 'datacol'.

SEQUENCE

  

It is an independent object in the databse; it is not tied to a particular column in a table. By default, it uses BIGINT as datatype. Specify a datatype you want otherwise. It supports MINVALUE and MAXVALUE within that type. It supports cycling but by default there is no cycle. 

  

START WITH

INCREMENT BY

  

default start value is minimum value of the datatype  used and default increment is 1.

In [25]:
use Northwinds2022TSQLV7
drop sequence dbo.SeqOrderIDs


Commands completed successfully.

Total execution time: 00:00:00.013

In [26]:
use Northwinds2022TSQLV7
CREATE SEQUENCE dbo.SeqOrderIDs AS INT 
	MINVALUE 1
	CYCLE;

--You can change any sequence properties except the datatype
ALTER SEQUENCE dbo.SeqOrderIDs
	NO CYCLE; --MINVAL, MAXVAL, RESTART WITH, INCREMENT BY, CYCLE/NO CYCLE, CACHE/NO CASHE

--To generate new sequence value:
SELECT NEXT VALUE FOR dbo.SeqOrderIDs


DROP TABLE IF EXISTS dbo.T1;
CREATE TABLE dbo.T1
(
	keycol INT NOT NULL
		CONSTRAINT PK_T1 PRIMARY KEY,
	datacol VARCHAR(10) NOT NULL 
);

(1 row affected)

Total execution time: 00:00:00.049

(No column name)
1


In [27]:

use Northwinds2022TSQLV7

--with	SEQUENCE, you can store the result of the function in a variable and  use it later in the code. You don't have to create a 
--new row to generate a new number.

DECLARE @newOrderId AS INT = NEXT VALUE FOR dbo.SeqOrderIDs;
INSERT INTO dbo.T1(keycol, datacol) VALUES (@neworderid, 'a');

SELECT * FROM dbo.T1 AS t

-- You can also place the command directly insider the INSERT.

INSERT INTO dbo.T1(keycol, datacol) VALUES (NEXT VALUE FOR dbo.SeqOrderIDs, 'b');

SELECT * FROM dbo.T1 AS t

--you can also generate a new sequence values in an UPDATE statement
UPDATE dbo.T1
	SET keycol = NEXT VALUE FOR dbo.SeqOrderIDs

SELECT * FROM dbo.T1 AS t

--to check the current state of the sequence, do the following:
SELECT current_value
FROM sys.sequences
WHERE OBJECT_ID = OBJECT_ID(N'dbo.SeqOrderIDs')

(1 row affected)

(1 row affected)

(1 row affected)

(2 rows affected)

(2 rows affected)

...

Total execution time: 00:00:00.061

keycol,datacol
2,a


keycol,datacol
2,a
3,b


keycol,datacol
4,a
5,b


current_value
5


SEQUENCE: Special Extension

  

NEXT VALUE FOR ... OVER enables you to control the order of the assigned sequence values.

In [29]:
use Northwinds2022TSQLV7
INSERT INTO dbo.T1(keycol, datacol)
	SELECT 
		NEXT VALUE FOR dbo.SeqOrderIDs OVER (ORDER BY hiredate),
		LEFT(Employeefirstname, 1) + LEFT(Employeelastname,1)
	FROM HumanResources.Employee AS e

SELECT * from dbo.T1;

(9 rows affected)

(20 rows affected)

Total execution time: 00:00:00.017

keycol,datacol
4,a
5,b
6,JL
7,SD
8,DF
9,YP
10,SM
11,PS
12,RK
13,MC


Using NEXT VALUE FOR function as a default constraint

In [ ]:
use Northwinds2022TSQLV7
ALTER TABLE dbo.T1 
	ADD CONSTRAINT DFT_T1_keycol
		DEFAULT (NEXT VALUE FOR dbo.SeqOrderIDs)
		FOR keycol;

INSERT INTO dbo.T1(datacol) VALUES ('C');
SELECT * FROM dbo.T1;

--to remove constraint, use DROP CONSTRAINT

Allocating a whole range of sequence values at once using Stored Procedure

  

SQL\_VARIANT is a generic data type that can hold within its various base data types.

In [31]:
DECLARE @first AS SQL_VARIANT;

EXEC sys.sp_sequence_get_range 
	@sequence_name = N'dbo.SeqOrderIDs',
	@range_size = 100,
	@range_first_value = @first OUTPUT;

SELECT @first 
--Each time you run, you 100 will be added to the @first.
--Sequence does not guarantee that there will be no gaps.

--clean up
DROP TABLE IF EXISTS dbo.T1;
DROP SEQUENCE IF EXISTS dbo.SeqOrderIds;

: Msg 208, Level 16, State 1, Procedure sys.sp_sequence_get_range_internal, Line 1
Invalid object name 'dbo.SeqOrderIDs'.

Total execution time: 00:00:00.008

Deleting Data

In [32]:
--Copying over tables
use Northwinds2022TSQLV7
DROP TABLE IF EXISTS dbo.Orders, dbo.Customers;

CREATE TABLE dbo.Customers
(
  custid       INT          NOT NULL,
  companyname  NVARCHAR(40) NOT NULL,
  contactname  NVARCHAR(30) NOT NULL,
  contacttitle NVARCHAR(30) NOT NULL,
  address      NVARCHAR(60) NOT NULL,
  city         NVARCHAR(15) NOT NULL,
  region       NVARCHAR(15) NULL,
  postalcode   NVARCHAR(10) NULL,
  country      NVARCHAR(15) NOT NULL,
  phone        NVARCHAR(24) NOT NULL,
  fax          NVARCHAR(24) NULL,
  CONSTRAINT PK_Customers PRIMARY KEY(custid)
);

CREATE TABLE dbo.Orders
(
  orderid        INT          NOT NULL,
  custid         INT          NULL,
  empid          INT          NOT NULL,
  shipperid      INT          NOT NULL,
  orderdate      DATE         NOT NULL,
  requireddate   DATE         NOT NULL,
  shippeddate    DATE         NULL,
 
  freight        MONEY        NOT NULL
    CONSTRAINT DFT_Orders_freight DEFAULT(0),
  shipname       NVARCHAR(40) NOT NULL,
  shipaddress    NVARCHAR(60) NOT NULL,
  shipcity       NVARCHAR(15) NOT NULL,
  shipregion     NVARCHAR(15) NULL,
  shippostalcode NVARCHAR(10) NULL,
  shipcountry    NVARCHAR(15) NOT NULL,
  userAuthenticationId INT NULL,
  DateAdded DATETIME2 NULL,
  dateoflastUpdate DATETIME2 NULL,
  CONSTRAINT PK_Orders PRIMARY KEY(orderid),
  CONSTRAINT FK_Orders_Customers FOREIGN KEY(custid)
    REFERENCES dbo.Customers(custid)
);
GO

INSERT INTO dbo.Customers SELECT * FROM Sales.Customer;
INSERT INTO dbo.Orders SELECT * FROM Sales.[ORDER];

Commands completed successfully.

(91 rows affected)

(830 rows affected)

Total execution time: 00:00:00.091

DELETE Statement

FROM

WHERE

Only the subset of rows for which the predicate evalutes to TRUE will be deleted. It is expensive because it is a fully logged operation.

In [33]:
use Northwinds2022TSQLV7
DELETE FROM dbo.Orders
WHERE orderdate < '20150101';



(152 rows affected)

Total execution time: 00:00:00.024

TRUNCATE

Deletes ALL rows from a table. It has no filter. It is minimally logged so it takes much less time than DELETE. Both DELETE and TRUNCATE are transactional. TRUNCATE resets the identiy value back to the original seed but DELETE does not. TRUNCATE statement is not allowed until you drop all foreign keys referencing the table with the ALTER TABLE DROP CONSTRAINT command. To avoid truncating a table by accident, you can create a dummy table with a foreign key pointing to ththat table. T-SQL also supports truncating certain partitions. Partitions allows importing data and purging historic data to be more efficient and has manageability purposes.

In [34]:
use Northwinds2022TSQLV7
TRUNCATE TABLE dbo.T1

--Syntax for partition
-- creating partitions
DROP TABLE IF EXISTS dbo.T1;
IF EXISTS (SELECT * FROM sys.partition_schemes WHERE name = N'PS1') DROP PARTITION SCHEME PS1;
IF EXISTS (SELECT * FROM sys.partition_functions WHERE name = N'PF1') DROP PARTITION FUNCTION PF1;

CREATE PARTITION FUNCTION PF1 (INT) AS RANGE LEFT FOR VALUES (10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120);
CREATE PARTITION SCHEME PS1 AS PARTITION PF1 ALL TO ([PRIMARY]);

CREATE TABLE dbo.T1
(
  keycol INT NOT NULL
    CONSTRAINT PK_T1 PRIMARY KEY,
  datacol INT NOT NULL
) ON PS1(keycol);
GO
--TRUNCATE
TRUNCATE TABLE dbo.T1 WITH (PARTITIONS (1, 3, 5, 7 TO 10));

--Clean up
DROP TABLE IF EXISTS dbo.T1;
IF EXISTS (SELECT * FROM sys.partition_schemes WHERE name = N'PS1') DROP PARTITION SCHEME PS1;
IF EXISTS (SELECT * FROM sys.partition_functions WHERE name = N'PF1') DROP PARTITION FUNCTION PF1;

: Msg 4701, Level 16, State 1, Line 2
Cannot find the object "T1" because it does not exist or you do not have permissions.

: Msg 4701, Level 16, State 1, Line 21
Cannot find the object "T1" because it does not exist or you do not have permissions.

Total execution time: 00:00:00.016

DELETE based on a join (nonstandard)

This is a nonstandard DELETE. You can delete rows from one table on a filter based on attributes from related rows from another table. In this case, you can delete rows from dbo.Orders based on an attribute (country) from dbo.Customers table. But it is better to stick to standard deletion using subquery.

In [ ]:
use Northwinds2022TSQLV7
DELETE FROM o 
FROM dbo.Orders AS o
	INNER JOIN dbo.Customers AS c
		ON o.custid = c.custid
WHERE c.country = N'USA';

-- o is the target of deletion.

DELETE using subquery (standard)

In [ ]:
use Northwinds2022TSQLV7
DELETE FROM dbo.Orders
WHERE EXISTS 
	(SELECT * 
	 FROM dbo.Customers AS C
	 WHERE Orders.custid = C.custid
		AND C.Country = N'USA');

Updating Data

In [35]:
--Copying over tables. 

use Northwinds2022TSQLV7
DROP TABLE IF EXISTS dbo.OrderDetails, dbo.Orders;

CREATE TABLE dbo.Orders
(
  orderid        INT          NOT NULL,
  custid         INT          NULL,
  empid          INT          NOT NULL,
  shipperid      INT          NOT NULL,
  orderdate      DATE         NOT NULL,
  requireddate   DATE         NOT NULL,
  shippeddate    DATE         NULL,
 
  freight        MONEY        NOT NULL
    CONSTRAINT DFT_Orders_freight DEFAULT(0),
  shipname       NVARCHAR(40) NOT NULL,
  shipaddress    NVARCHAR(60) NOT NULL,
  shipcity       NVARCHAR(15) NOT NULL,
  shipregion     NVARCHAR(15) NULL,
  shippostalcode NVARCHAR(10) NULL,
  shipcountry    NVARCHAR(15) NOT NULL,
  userAuthenticationId INT NULL,
  DateAdded DATETIME2 NULL,
  dateoflastUpdate DATETIME2 NULL,
  CONSTRAINT PK_Orders PRIMARY KEY(orderid),
  CONSTRAINT FK_Orders_Customers FOREIGN KEY(custid)
    REFERENCES dbo.Customers(custid)
);

CREATE TABLE dbo.OrderDetails
(
  orderid   INT           NOT NULL,
  productid INT           NOT NULL,
  unitprice MONEY         NOT NULL
    CONSTRAINT DFT_OrderDetails_unitprice DEFAULT(0),
  qty       SMALLINT      NOT NULL
    CONSTRAINT DFT_OrderDetails_qty DEFAULT(1),
  discount  NUMERIC(4, 3) NOT NULL
    CONSTRAINT DFT_OrderDetails_discount DEFAULT(0),
  CONSTRAINT PK_OrderDetails PRIMARY KEY(orderid, productid),
  CONSTRAINT FK_OrderDetails_Orders FOREIGN KEY(orderid)
    REFERENCES dbo.Orders(orderid),
  CONSTRAINT CHK_discount  CHECK (discount BETWEEN 0 AND 1),
  CONSTRAINT CHK_qty  CHECK (qty > 0),
  CONSTRAINT CHK_unitprice CHECK (unitprice >= 0)
);
GO

INSERT INTO dbo.Orders SELECT * FROM Sales.[ORDER];
INSERT INTO dbo.OrderDetails SELECT * FROM Sales.OrderDetail;

Commands completed successfully.

(830 rows affected)

(2155 rows affected)

Total execution time: 00:00:00.089

**UPDATE statement**

UPDATE

SET

WHERE

You may use the following compound assignment operators in the SET clause: +=, -=, /=, \*=, %=

In [ ]:
use Northwinds2022TSQLV7
UPDATE dbo.orderDetails
	SET discount = discount + 0.05
WHERE productid = 51;

--Keep in mind all-at-once operations. Let's say col1 is originally 100. The values of col1 and col2 will both be 110 because the value before the update will be applied to both regardless of order.
UPDATE dbo.T1 
	SET col1 = col1 + 10, col2 = col1 + 10;

--because of all-at-once operation, you do not have to have a temp variable to store value before swapping values of two variables.
--For both operation, the original value will be used.
UPDATE dbo.T1 
	SET col1=col2, col2=col1;

UPDATE based on a join (nonstandard)

You can update based on an attribute from another table.

Proposition: Increase the discount of all order details of orders placed by customer 1 by 5 percent.

In [ ]:
use Northwinds2022TSQLV7
UPDATE od 
	SET discount += 0.05
FROM dbo.OrderDetails AS OD
	INNER JOIN dbo.Orders AS O
		ON OD.orderid = o.orderId
WHERE o.custid = 1;



JOIN using subquery (standard)

In [ ]:
use Northwinds2022TSQLV7
UPDATE dbo.OrderDetails
	SET discount+= 0.05
WHERE EXISTS (SELECT * FROM dbo.orders AS o
			  WHERE OrderDetails.orderid = o.orderid
			  AND o.custid = 1)

<span style="color: #608b4e;">It&nbsp;is&nbsp;best&nbsp;to&nbsp;use&nbsp;the&nbsp;standard&nbsp;subquery&nbsp;to&nbsp;update.&nbsp;At&nbsp;times,&nbsp;it&nbsp;is&nbsp;easier&nbsp;to&nbsp;update using join. You need separate subqueries for filtering and assignment and sa separate subquery for each assignment.</span>

<span style="color: #608b4e;">Compare and contrast the following update using join and subquery.</span>

In [ ]:
DROP TABLE IF EXISTS dbo.T1, dbo.T2;

CREATE TABLE dbo.T1
(
  keycol INT NOT NULL
    CONSTRAINT PK_T1 PRIMARY KEY,
  col1 INT NOT NULL,
  col2 INT NOT NULL,
  col3 INT NOT NULL,
  col4 VARCHAR(10) NOT NULL
);

CREATE TABLE dbo.T2
(
  keycol INT NOT NULL
    CONSTRAINT PK_T2 PRIMARY KEY,
  col1 INT NOT NULL,
  col2 INT NOT NULL,
  col3 INT NOT NULL,
  col4 VARCHAR(10) NOT NULL
);
GO

In [ ]:
--JOIN
UPDATE t1 
	SET col1 = T2.col1,
		col2 = T2.col2,
		col3 = T2.col3
FROM dbo.T1 
	JOIN dbo.T2
		ON T2.keycol = T1.keycol
WHERE T2.col4 = 'ABC'

--The same query using the subqueries
UPDATE dbo.T1 
	SET col1 = (SELECT col1
				FROM dbo.T2
				WHERE T2.keycol = T1.keycol),
		col2 = (SELECT col2
				FROM dbo.T2
				WHERE T2.keycol = T1.keycol),
		col3 = (SELECT col3
				FROM dbo.T2
				WHERE T2.keycol = T1.keycol)
WHERE EXISTS
	(SELECT *
	 FROM dbo.T2
	 WHERE T2.keycol = T1.keycol
		AND T2.col4 = 'ABC'); 

--Row Constructors is a subquery that enables multiple assignment. This one is a bit more simple than the one above but it still requires separte subqueries for filtering and for assignment.
UPDATE dbo.T1
	SET (col1, col2, col3) =
		(SELECT col1, col2, col3
		 FROM dbo.T2
		 WHERE T2.keycol = T1.keycol)
	WHERE EXISTS 
		(SELECT *
		 FROM dbo.T2
		 WHERE T2.keycol = T1.keycol
			AND T2.col4 = 'ABC')


Assignment UPDATE

  

You can both update data in a table and assign values to variables at the same time. Saves you the need to use deparate UPDATE and SELECT statements. 

  

Proposition: Guarauntee no gaps between the values by maintaining a custom sequence/autonumbering mecahnism other than identity property and sequence object.

In [37]:
use Northwinds2022TSQLV7
DROP TABLE IF EXISTS dbo.MySequences;

CREATE TABLE dbo.MySequences
(
	id VARCHAR(10) NOT NULL
		CONSTRAINT PK_MySequences PRIMARY KEY (id),
	val INT NOT NULL 
);

INSERT INTO dbo.MySequences VALUES ('SEQ1', 0);

DECLARE @nextval AS INT;

UPDATE dbo.MySequences
	SET @nextval = val +=1
WHERE id  = 'SEQ1';

SELECT @nextval;

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.042

(No column name)
1


Merging data

Merge data from source into a target applying insert, update, and delete based on conditional logic.

In [40]:
use Northwinds2022TSQLV7

DROP TABLE IF EXISTS dbo.Customer, dbo.CustomersStage;
GO

CREATE TABLE dbo.Customer
(
  custid      INT         NOT NULL,
  companyname VARCHAR(25) NOT NULL,
  phone       VARCHAR(20) NOT NULL,
  address     VARCHAR(50) NOT NULL,
  CONSTRAINT PK_Customer PRIMARY KEY(custid)
);

INSERT INTO dbo.Customer(custid, companyname, phone, address)
VALUES
  (1, 'cust 1', '(111) 111-1111', 'address 1'),
  (2, 'cust 2', '(222) 222-2222', 'address 2'),
  (3, 'cust 3', '(333) 333-3333', 'address 3'),
  (4, 'cust 4', '(444) 444-4444', 'address 4'),
  (5, 'cust 5', '(555) 555-5555', 'address 5');

CREATE TABLE dbo.CustomersStage
(
  custid      INT         NOT NULL,
  companyname VARCHAR(25) NOT NULL,
  phone       VARCHAR(20) NOT NULL,
  address     VARCHAR(50) NOT NULL,
  CONSTRAINT PK_CustomersStage PRIMARY KEY(custid)
);

INSERT INTO dbo.CustomersStage(custid, companyname, phone, address)
VALUES
  (2, 'AAAAA', '(222) 222-2222', 'address 2'),
  (3, 'cust 3', '(333) 333-3333', 'address 3'),
  (5, 'BBBBB', 'CCCCC', 'DDDDD'),
  (6, 'cust 6 (new)', '(666) 666-6666', 'address 6'),
  (7, 'cust 7 (new)', '(777) 777-7777', 'address 7');

 SELECT * from dbo.Customer;
 SELECT * from  dbo.CustomersStage;

MERGE INTO dbo.Customer AS TGT
USING dbo.CustomersStage AS SRC
	ON TGT.custid = SRC.custid
WHEN MATCHED THEN 
	UPDATE SET
		TGT.companyname = SRC.companyname,
		TGT.phone = SRC.phone,
		TGT.[ADDRESS] = SRC.[address]
	WHEN NOT MATCHED THEN 
		INSERT (custid, companyname, phone, address)
		VALUES (SRC.custid, SRC.companyname, SRC.phone, SRC.address);

SELECT * FROM dbo.Customer;

--the following will delete the 1st and 4th rows for they were only present in the first table, not the second.
MERGE INTO dbo.Customer AS TGT
USING dbo.CustomersStage AS SRC
	ON TGT.custid = SRC.custid
WHEN MATCHED THEN 
	UPDATE SET
		TGT.companyname = SRC.companyname,
		TGT.phone = SRC.phone,
		TGT.[ADDRESS] = SRC.[address]
	WHEN NOT MATCHED THEN 
		INSERT (custid, companyname, phone, address)
		VALUES (SRC.custid, SRC.companyname, SRC.phone, SRC.address)
	WHEN NOT MATCHED BY SOURCE THEN 
		DELETE;

SELECT * FROM dbo.Customer;

Commands completed successfully.

(5 rows affected)

(5 rows affected)

(5 rows affected)

(5 rows affected)

...

Total execution time: 00:00:00.102

custid,companyname,phone,address
1,cust 1,(111) 111-1111,address 1
2,cust 2,(222) 222-2222,address 2
3,cust 3,(333) 333-3333,address 3
4,cust 4,(444) 444-4444,address 4
5,cust 5,(555) 555-5555,address 5


custid,companyname,phone,address
2,AAAAA,(222) 222-2222,address 2
3,cust 3,(333) 333-3333,address 3
5,BBBBB,CCCCC,DDDDD
6,cust 6 (new),(666) 666-6666,address 6
7,cust 7 (new),(777) 777-7777,address 7


custid,companyname,phone,address
1,cust 1,(111) 111-1111,address 1
2,AAAAA,(222) 222-2222,address 2
3,cust 3,(333) 333-3333,address 3
4,cust 4,(444) 444-4444,address 4
5,BBBBB,CCCCC,DDDDD
6,cust 6 (new),(666) 666-6666,address 6
7,cust 7 (new),(777) 777-7777,address 7


custid,companyname,phone,address
2,AAAAA,(222) 222-2222,address 2
3,cust 3,(333) 333-3333,address 3
5,BBBBB,CCCCC,DDDDD
6,cust 6 (new),(666) 666-6666,address 6
7,cust 7 (new),(777) 777-7777,address 7


Merging Data - Additional condition

  

You can add WHEN MATCHED AND clause to check that at least one of the column values is idfferent to justify the update ACTION.

In [42]:
use Northwinds2022TSQLV7

MERGE dbo.Customer AS TGT
USING dbo.CustomersStage AS SRC
  ON TGT.custid = SRC.custid
WHEN MATCHED AND 
       (   TGT.companyname <> SRC.companyname
        OR TGT.phone       <> SRC.phone
        OR TGT.address     <> SRC.address) THEN
  UPDATE SET
    TGT.companyname = SRC.companyname,
    TGT.phone = SRC.phone,
    TGT.address = SRC.address
WHEN NOT MATCHED THEN 
  INSERT (custid, companyname, phone, address)
  VALUES (SRC.custid, SRC.companyname, SRC.phone, SRC.address);

select * from dbo.Customer

(0 rows affected)

(5 rows affected)

Total execution time: 00:00:00.007

custid,companyname,phone,address
2,AAAAA,(222) 222-2222,address 2
3,cust 3,(333) 333-3333,address 3
5,BBBBB,CCCCC,DDDDD
6,cust 6 (new),(666) 666-6666,address 6
7,cust 7 (new),(777) 777-7777,address 7


```
 6
Run the following code to create the tables Orders and OrderDetails and populate them with data

```
```
You have to drop all FK relationships before truncating table. In the following two tables, there is only one FK in OrderDetails table. You drop that then truncate. Add the foreign key relationship back. 

```

<span class="c1" style="color: rgb(64, 128, 128); font-style: italic;"><br></span>

In [ ]:
DROP TABLE IF EXISTS dbo.OrderDetails, dbo.Orders;

CREATE TABLE dbo.Orders
(
  orderid        INT          NOT NULL,
  custid         INT          NULL,
  empid          INT          NOT NULL,
  shipperid      INT          NOT NULL,
  orderdate      DATE         NOT NULL,
  requireddate   DATE         NOT NULL,
  shippeddate    DATE         NULL,
  
  freight        MONEY        NOT NULL
    CONSTRAINT DFT_Orders_freight DEFAULT(0),
  shipname       NVARCHAR(40) NOT NULL,
  shipaddress    NVARCHAR(60) NOT NULL,
  shipcity       NVARCHAR(15) NOT NULL,
  shipregion     NVARCHAR(15) NULL,
  shippostalcode NVARCHAR(10) NULL,
  shipcountry    NVARCHAR(15) NOT NULL,
  userAuthenticationid int NULL,
  datadded datetime2 NULL,
  dateoflastupdate DATETIME2 NULL,
  CONSTRAINT PK_Orders PRIMARY KEY(orderid)
);

SELECT * FROM sales.[Order] AS o

CREATE TABLE dbo.OrderDetails
(
  orderid   INT           NOT NULL,
  productid INT           NOT NULL,
  unitprice MONEY         NOT NULL
    CONSTRAINT DFT_OrderDetails_unitprice DEFAULT(0),
  qty       SMALLINT      NOT NULL
    CONSTRAINT DFT_OrderDetails_qty DEFAULT(1),
  discount  NUMERIC(4, 3) NOT NULL
    CONSTRAINT DFT_OrderDetails_discount DEFAULT(0),
  CONSTRAINT PK_OrderDetails PRIMARY KEY(orderid, productid),
  CONSTRAINT FK_OrderDetails_Orders FOREIGN KEY(orderid)
    REFERENCES dbo.Orders(orderid),
  CONSTRAINT CHK_discount  CHECK (discount BETWEEN 0 AND 1),
  CONSTRAINT CHK_qty  CHECK (qty > 0),
  CONSTRAINT CHK_unitprice CHECK (unitprice >= 0)
);
GO

INSERT INTO dbo.Orders SELECT * FROM Sales.[ORDER];
INSERT INTO dbo.OrderDetails SELECT * FROM Sales.OrderDetail;

ALTER TABLE dbo.OrderDetails DROP CONSTRAINT FK_OrderDetails_Orders;

TRUNCATE TABLE dbo.OrderDetails;
TRUNCATE TABLE dbo.orders;

ALTER TABLE dbo.OrderDetails
	ADD CONSTRAINT FK_OrderDetails_Orders
		FOREIGN KEY (orderId) REFERENCES dbo.Orders(orderId);